In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import json
import demjson
import re
import uuid
import mongoengine as mdb
from datetime import datetime

In [ ]:
source_path = "C:\source\data\scraped\stadium_goods\original-details.csv"
path_to_save = "C:\source\data\scraped\stadium_goods"
addt_source_path = "C:\source\data\scraped\stadium_goods\categories.csv"

In [79]:
def to_dict(data):
    return [r.to_mongo().to_dict() for r in data]
def to_dataframe(data):
    dicts = to_dict(data)
    fields = list(dicts[0].keys())
    return pd.DataFrame(to_dict(data), columns=fields[1:])

In [42]:
mdb.disconnect()
mdb.connect(
    db="sneakerResaleDB",
    host="mongodb://db.kicksware.com:443",
    username="root",
    password="greenJordans",
    authentication_source="admin",
    ssl=True,
    ssl_ca_certs="/source/certs/mongo/ca.pem"
)


class SneakerBrand(mdb.Document):
    unique_id = mdb.StringField(db_field="uniqueid")
    name = mdb.StringField()
    logo = mdb.StringField()
    hero = mdb.StringField()
    description = mdb.StringField()
    meta = {"collection": "brands"}


class SneakerModel(mdb.Document):
    unique_id = mdb.StringField(db_field="uniqueid")
    name = mdb.StringField()
    hero = mdb.StringField()
    description = mdb.StringField()
    meta = {"collection": "models"}


class SneakerReference(mdb.Document):
    unique_id = mdb.StringField(db_field="uniqueid")
    manufacture_sku = mdb.StringField(db_field="manufacturesku")
    brand_name = mdb.StringField(db_field="brandname")
    model_name = mdb.StringField(db_field="modelname")
    base_model_name = mdb.StringField(db_field="basemodelname")
    brand =  mdb.StringField()
    model =  mdb.StringField()
    basemodel = mdb.StringField()
    description = mdb.StringField()
    release_date = mdb.DateField("releasedate")
    release_strdate = mdb.StringField()
    color = mdb.StringField()
    gender = mdb.StringField()
    nickname = mdb.StringField()
    price = mdb.DecimalField()
    materials = mdb.ListField()
    categories = mdb.ListField()
    image_link = mdb.StringField(db_field="imagelink")
    image_links = mdb.ListField(db_field="imagelinks")
    stadium_url = mdb.StringField(db_field="stadiumurl")
    meta = { "collection": "references", 'strict': False }

In [92]:
df = to_dataframe(SneakerReference.objects())
print(f"Initial: {len(df)}")
pd.set_option("max_rows", 200)

Initial: 6862


In [93]:
gk = df.groupby("uniqueid")
df = gk.first()
print(f"Distinct: {len(df)}")

Distinct: 6855


In [81]:
pd.unique(df["brand"].values)

array(['Ewing', 'Jordan', 'Nike', 'Adidas', 'Reebok', 'Asics', 'Puma',
       'New-Balance', 'Fila', 'Converse', 'Vans', 'Diadora', 'Saucony',
       'Under-Armour', 'Timberland', 'Clarks', 'Bape', 'GREATS',
       'Diamond-Supply', 'KARHU', 'Revenge-X-Storm',
       'Information-Technology', 'Ice-Cream', 'Footwear', 'Yeezy',
       'Dr-Martens', 'Q4-Sports', 'Off-White', 'GOLDEN-GOOSE',
       'Fear-Of-God'], dtype=object)

Filter data:

In [82]:
keywords_query = '''Pant|Pants|Coat|Shirt|Reverse|Jacket|Crew|Tee|Pullover|Strapback|Hat|Short|Bottle|Sock|Socks|Hoodie|Beanie|Bomber|Sleeveless|Sweatpants|Sleeve|Sweatshirt|Overshirt|Bandana|Bearbrick|Bag|Belt|Poncho|Parka|Case|Bodysuit|Sunglasses|Glasses|Dress|Stitch|Jeans|Leggings|Lock|Luggage|Gilet|Wipes|Essential Kit|Sneaker Box|Wallet|Chain|Stone Island|Swim|Pullove|Famous|Pablo|Romantic|Jersey Top|Tank Top|Track Top|Anorak|Joggi|Zimmermann|Cubs|Cushion|Flower|Football|T-Sh|Kimono|Astroworld|Pajama|Pocket|Knife|Organizer|Pocket|Purse|Look|Basebal|Bodysuit|Crew|Crewneck|Shoulder|Sweater|Camper|Camera|Holder|Card|Strapback|Denim|Body|Hands|Zippy|Script|Swea|Embroidered|Heavyweigh|Neck|Pillow|Hooded|Striped|Cotton|HAT|Button|Manifestation|Collectible|Real Men|Pullover|Balaclava|Skateboard|Popsockets|Polo|Head|Full Zip|Hoo|HOODIE|JACKET'''
brand_query = [
    "Ewing", "Jordan", "Nike", "Adidas", "Reebok", "Asics", "Puma",
    "New Balance", "Fila", "Converse", "Vans", "Diadora",
    "Saucony", "Under Armour", "Timberland", "Clarks",
    "Bape", "GREATS", "Diamond Supply", "KARHU", "Revenge X Storm",
    "Information Technology", "Ice Cream", "Footwear", "Yeezy", "Kryptonite",
    "Dr Martens", "Q4 Sports", "Off-White", "Kanye West",
    "Anti Social Social Club", "GOLDEN GOOSE", "Fear Of God"
]

In [97]:
print(len(df))
qdf = df[df["brandname"].isin(brand_query)]
print(len(qdf))

#gdf["brand"] = gdf["brand"].map({"Footwear": "Nike"}).fillna(gdf["brand"]) #todo
pd.unique(qdf["brand"].values)
qdf

6855
6855


,manufacturesku,brandname,modelname,basemodelname,brand,model,description,color,gender,nickname,price,materials,categories,imagelink,imagelinks,stadiumurl
uniqueid,,,,,,,,,,,,,,,,
1-2-cent_344646-600,344646 600,Nike,1/2 Cent,,Nike,Nike_1-2-Cent,The Nike 1/2 cent is a mid/high top basketball...,VARSITY RED/BLACK-VARSITY RED,Mens,Cranberry,0.0,[Synthetics],[Penny Hardaway],1-2-cent_344646-600.png,"[1-2-cent_344646-600_2.png, 1-2-cent_344646-60...",https://www.stadiumgoods.com/nike-1-2-cent-cra...
101-backless-sneaker-all-over-_6h19-7016-sca-105,6H19-7016-SCA-105,Fear Of God,101 BACKLESS SNEAKER ALL OVER,,Fear-Of-God,Fear-Of-God_101-BACKLESS-SNEAKER-ALL-OVER-,101 BACKLESS SNEAKER ALL OVER \n6H19-7016-SCA-105,BLACK/BONE,Mens,,410.0,[],[],101-backless-sneaker-all-over-_6h19-7016-sca-1...,[101-backless-sneaker-all-over-_6h19-7016-sca-...,https://www.stadiumgoods.com/fear-of-god-101-b...
101-lace-up-sneaker-print_6h19-7000-sca-108,6H19-7000-SCA-108,Fear Of God,101 LACE UP SNEAKER PRINT,,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER-PRINT,101 LACE UP SNEAKER PRINT\n6H19 7000 SCA 108,BONE/CREAM PRINT,Mens,,430.0,[],[],101-lace-up-sneaker-print_6h19-7000-sca-108.png,[101-lace-up-sneaker-print_6h19-7000-sca-108_2...,https://www.stadiumgoods.com/fear-of-god-101-l...
101-lace-up-sneaker_6f19-7000-nca-966,6F19-7000-NCA-966,Fear Of God,101 LACE UP SNEAKER,,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER,101 LACE UP SNEAKER\n6F19-7000-NCA-966,BLACK CREAM,Mens,,460.0,[],[],101-lace-up-sneaker_6f19-7000-nca-966.png,"[101-lace-up-sneaker_6f19-7000-nca-966_2.png, ...",https://www.stadiumgoods.com/fear-of-god-101-l...
101-lace-up-sneaker_6h19-7000-can-300,6H19-7000-CAN-300,Fear Of God,101 LACE UP SNEAKER,,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER,101 LACE UP SNEAKER\n6H19 7000 CAN 300,GREEN,Mens,,430.0,[],[],101-lace-up-sneaker_6h19-7000-can-300.png,"[101-lace-up-sneaker_6h19-7000-can-300_2.png, ...",https://www.stadiumgoods.com/fear-of-god-101-l...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zx-flux_s32277,S32277,Adidas,ZX Flux,,Adidas,Adidas_ZX-Flux,"In 2015, the adidas ZX Flux returned with an a...",FTWWHT/FTWWHT/CLGREY,Mens,,0.0,[],[],zx-flux_s32277.png,"[zx-flux_s32277_2.png, zx-flux_s32277_3.png, z...",https://www.stadiumgoods.com/adidas-zx-flux-s3...
zx-flux_s32278,S32278,Adidas,ZX Flux,,Adidas,Adidas_ZX-Flux,The Adidas ZX Flux serves up a sleek remix of ...,POWRED/POWRED/CBURGU,Mens,,70.0,[],[],zx-flux_s32278.png,"[zx-flux_s32278_2.png, zx-flux_s32278_3.png, z...",https://www.stadiumgoods.com/adidas-zx-flux-s3...
zx-tr-mid_d69375,D69375,Adidas,ZX TR Mid,,Adidas,Adidas_ZX-TR-Mid,From the “Vanguard” collection by Extra Butter...,DRKCAR/SUPCOL/OWHITE,Mens,Extra Butter,0.0,"[Fabric, Suede]","[More Adidas Shoes, ZX Flux, eBay]",zx-tr-mid_d69375.png,"[zx-tr-mid_d69375_2.png, zx-tr-mid_d69375_3.png]",https://www.stadiumgoods.com/adidas-zx-tr-mid-...


In [98]:
qdf = qdf[~qdf["modelname"].str.contains(keywords_query)]
print(len(qdf))
qdf # filtered by keywords

6855


,manufacturesku,brandname,modelname,basemodelname,brand,model,description,color,gender,nickname,price,materials,categories,imagelink,imagelinks,stadiumurl
uniqueid,,,,,,,,,,,,,,,,
1-2-cent_344646-600,344646 600,Nike,1/2 Cent,,Nike,Nike_1-2-Cent,The Nike 1/2 cent is a mid/high top basketball...,VARSITY RED/BLACK-VARSITY RED,Mens,Cranberry,0.0,[Synthetics],[Penny Hardaway],1-2-cent_344646-600.png,"[1-2-cent_344646-600_2.png, 1-2-cent_344646-60...",https://www.stadiumgoods.com/nike-1-2-cent-cra...
101-backless-sneaker-all-over-_6h19-7016-sca-105,6H19-7016-SCA-105,Fear Of God,101 BACKLESS SNEAKER ALL OVER,,Fear-Of-God,Fear-Of-God_101-BACKLESS-SNEAKER-ALL-OVER-,101 BACKLESS SNEAKER ALL OVER \n6H19-7016-SCA-105,BLACK/BONE,Mens,,410.0,[],[],101-backless-sneaker-all-over-_6h19-7016-sca-1...,[101-backless-sneaker-all-over-_6h19-7016-sca-...,https://www.stadiumgoods.com/fear-of-god-101-b...
101-lace-up-sneaker-print_6h19-7000-sca-108,6H19-7000-SCA-108,Fear Of God,101 LACE UP SNEAKER PRINT,,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER-PRINT,101 LACE UP SNEAKER PRINT\n6H19 7000 SCA 108,BONE/CREAM PRINT,Mens,,430.0,[],[],101-lace-up-sneaker-print_6h19-7000-sca-108.png,[101-lace-up-sneaker-print_6h19-7000-sca-108_2...,https://www.stadiumgoods.com/fear-of-god-101-l...
101-lace-up-sneaker_6f19-7000-nca-966,6F19-7000-NCA-966,Fear Of God,101 LACE UP SNEAKER,,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER,101 LACE UP SNEAKER\n6F19-7000-NCA-966,BLACK CREAM,Mens,,460.0,[],[],101-lace-up-sneaker_6f19-7000-nca-966.png,"[101-lace-up-sneaker_6f19-7000-nca-966_2.png, ...",https://www.stadiumgoods.com/fear-of-god-101-l...
101-lace-up-sneaker_6h19-7000-can-300,6H19-7000-CAN-300,Fear Of God,101 LACE UP SNEAKER,,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER,101 LACE UP SNEAKER\n6H19 7000 CAN 300,GREEN,Mens,,430.0,[],[],101-lace-up-sneaker_6h19-7000-can-300.png,"[101-lace-up-sneaker_6h19-7000-can-300_2.png, ...",https://www.stadiumgoods.com/fear-of-god-101-l...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zx-flux_s32277,S32277,Adidas,ZX Flux,,Adidas,Adidas_ZX-Flux,"In 2015, the adidas ZX Flux returned with an a...",FTWWHT/FTWWHT/CLGREY,Mens,,0.0,[],[],zx-flux_s32277.png,"[zx-flux_s32277_2.png, zx-flux_s32277_3.png, z...",https://www.stadiumgoods.com/adidas-zx-flux-s3...
zx-flux_s32278,S32278,Adidas,ZX Flux,,Adidas,Adidas_ZX-Flux,The Adidas ZX Flux serves up a sleek remix of ...,POWRED/POWRED/CBURGU,Mens,,70.0,[],[],zx-flux_s32278.png,"[zx-flux_s32278_2.png, zx-flux_s32278_3.png, z...",https://www.stadiumgoods.com/adidas-zx-flux-s3...
zx-tr-mid_d69375,D69375,Adidas,ZX TR Mid,,Adidas,Adidas_ZX-TR-Mid,From the “Vanguard” collection by Extra Butter...,DRKCAR/SUPCOL/OWHITE,Mens,Extra Butter,0.0,"[Fabric, Suede]","[More Adidas Shoes, ZX Flux, eBay]",zx-tr-mid_d69375.png,"[zx-tr-mid_d69375_2.png, zx-tr-mid_d69375_3.png]",https://www.stadiumgoods.com/adidas-zx-tr-mid-...


In [99]:
# handle NaN values
qdf = qdf.replace(np.nan, '', regex=True)
len(qdf)

6855

Data transformation:

By catgories:

In [105]:
def camel_case_split(s):
    idx = list(map(str.isupper, s))
    l = [0]
    for (i, (x, y)) in enumerate(zip(idx, idx[1:])):
        if x and not y:
            l.append(i)
        elif not x and y:
            l.append(i+1)
    l.append(len(s))
    return [s[x:y] for x, y in zip(l, l[1:]) if x < y]
    
def get_acronym(source):
    return "".join(filter(str.isupper, source.title()))

def determine_base_model(row):
    model = row["modelname"]
    brand = row["brandname"]
    categories = row["categories"]
    if not categories:
        return None
    distances = {}
    for category in categories:
        distance = sum([word.lower() in model.lower() for word in camel_case_split(category)])
        if not distance:
                distance = 1 if get_acronym(category) in model else 0
        if distance:
            distances[category] = distance
    
    base_model = max(distances or [None], key=distances.get)
    return base_model

In [108]:
df["basemodelname"] = df.apply(determine_base_model, axis=1)
df

,manufacturesku,brandname,modelname,basemodelname,brand,model,description,color,gender,nickname,price,materials,categories,imagelink,imagelinks,stadiumurl,BaseModel
uniqueid,,,,,,,,,,,,,,,,,
1-2-cent_344646-600,344646 600,Nike,1/2 Cent,None,Nike,Nike_1-2-Cent,The Nike 1/2 cent is a mid/high top basketball...,VARSITY RED/BLACK-VARSITY RED,Mens,Cranberry,0.0,[Synthetics],[Penny Hardaway],1-2-cent_344646-600.png,"[1-2-cent_344646-600_2.png, 1-2-cent_344646-60...",https://www.stadiumgoods.com/nike-1-2-cent-cra...,None
101-backless-sneaker-all-over-_6h19-7016-sca-105,6H19-7016-SCA-105,Fear Of God,101 BACKLESS SNEAKER ALL OVER,None,Fear-Of-God,Fear-Of-God_101-BACKLESS-SNEAKER-ALL-OVER-,101 BACKLESS SNEAKER ALL OVER \n6H19-7016-SCA-105,BLACK/BONE,Mens,,410.0,[],[],101-backless-sneaker-all-over-_6h19-7016-sca-1...,[101-backless-sneaker-all-over-_6h19-7016-sca-...,https://www.stadiumgoods.com/fear-of-god-101-b...,None
101-lace-up-sneaker-print_6h19-7000-sca-108,6H19-7000-SCA-108,Fear Of God,101 LACE UP SNEAKER PRINT,None,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER-PRINT,101 LACE UP SNEAKER PRINT\n6H19 7000 SCA 108,BONE/CREAM PRINT,Mens,,430.0,[],[],101-lace-up-sneaker-print_6h19-7000-sca-108.png,[101-lace-up-sneaker-print_6h19-7000-sca-108_2...,https://www.stadiumgoods.com/fear-of-god-101-l...,None
101-lace-up-sneaker_6f19-7000-nca-966,6F19-7000-NCA-966,Fear Of God,101 LACE UP SNEAKER,None,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER,101 LACE UP SNEAKER\n6F19-7000-NCA-966,BLACK CREAM,Mens,,460.0,[],[],101-lace-up-sneaker_6f19-7000-nca-966.png,"[101-lace-up-sneaker_6f19-7000-nca-966_2.png, ...",https://www.stadiumgoods.com/fear-of-god-101-l...,None
101-lace-up-sneaker_6h19-7000-can-300,6H19-7000-CAN-300,Fear Of God,101 LACE UP SNEAKER,None,Fear-Of-God,Fear-Of-God_101-LACE-UP-SNEAKER,101 LACE UP SNEAKER\n6H19 7000 CAN 300,GREEN,Mens,,430.0,[],[],101-lace-up-sneaker_6h19-7000-can-300.png,"[101-lace-up-sneaker_6h19-7000-can-300_2.png, ...",https://www.stadiumgoods.com/fear-of-god-101-l...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zx-flux_s32277,S32277,Adidas,ZX Flux,None,Adidas,Adidas_ZX-Flux,"In 2015, the adidas ZX Flux returned with an a...",FTWWHT/FTWWHT/CLGREY,Mens,,0.0,[],[],zx-flux_s32277.png,"[zx-flux_s32277_2.png, zx-flux_s32277_3.png, z...",https://www.stadiumgoods.com/adidas-zx-flux-s3...,None
zx-flux_s32278,S32278,Adidas,ZX Flux,None,Adidas,Adidas_ZX-Flux,The Adidas ZX Flux serves up a sleek remix of ...,POWRED/POWRED/CBURGU,Mens,,70.0,[],[],zx-flux_s32278.png,"[zx-flux_s32278_2.png, zx-flux_s32278_3.png, z...",https://www.stadiumgoods.com/adidas-zx-flux-s3...,None
zx-tr-mid_d69375,D69375,Adidas,ZX TR Mid,ZX Flux,Adidas,Adidas_ZX-TR-Mid,From the “Vanguard” collection by Extra Butter...,DRKCAR/SUPCOL/OWHITE,Mens,Extra Butter,0.0,"[Fabric, Suede]","[More Adidas Shoes, ZX Flux, eBay]",zx-tr-mid_d69375.png,"[zx-tr-mid_d69375_2.png, zx-tr-mid_d69375_3.png]",https://www.stadiumgoods.com/adidas-zx-tr-mid-...,ZX Flux


By JSON base model list:

In [115]:
kdf = pd.read_json("../meta/base-model-tags.json")
base_models = kdf[0].tolist()
for i, model in enumerate(base_models): # validity check
    if any([model in lower for lower in base_models[i+1:]]):
        print(model)

In [113]:
s = df.modelname.str.len().sort_values(ascending=False).index
sdf = df.reindex(s)
sequence = list(sdf.T.to_dict().values())
groups = {}
for model in base_models:
    child_models = [item for item in sequence if model.upper() in item["modelname"].upper()]
    [sequence.remove(item) for item in child_models]
    groups[model] = child_models
group_analit = { key: len(val) for key, val in groups.items() }

In [114]:
sequence = list()
for key, items in groups.items():
    sequence.extend([dict(item, basemodel=key) for item in items])
gdf = pd.DataFrame(sequence)
gdf

,manufacturesku,brandname,modelname,basemodelname,brand,model,description,color,gender,nickname,price,materials,categories,imagelink,imagelinks,stadiumurl,BaseModel,basemodel
0,785563 154,Jordan,Air Jordan 15 x PSNY Womens Bo,Jordan,Jordan,Jordan_Air-Jordan-15-x-PSNY-Womens-Bo,Air Jordan 15 x PSNY Womens Bo\r\n785563 154,BLACK/WHITE-WHITE,Womens,Public School,785.0,[Leather],"[15 to Present, Jordan, Women]",air-jordan-15-x-psny-womens-bo_785563-154.png,[air-jordan-15-x-psny-womens-bo_785563-154_2.p...,https://www.stadiumgoods.com/jordan-air-jordan...,Jordan,Jordan 15
1,AO2568 200,Jordan,Air Jordan 15 Retro WVN PSNY,Jordan,Jordan,Jordan_Air-Jordan-15-Retro-WVN-PSNY,The good folks at Public School return with th...,MEDIUM OLIVE/MEDIUM OLIVE-SAIL,Mens,PSNY,239.0,"[Rubber, Synthetics]","[15 to Present, Jordan, Gifts]",air-jordan-15-retro-wvn-psny_ao2568-200.png,[air-jordan-15-retro-wvn-psny_ao2568-200_2.png...,https://www.stadiumgoods.com/jordan-air-jordan...,Jordan,Jordan 15
2,921194 011 FF,Jordan,Air Jordan 15 Retro Woven,None,Jordan,Jordan_Air-Jordan-15-Retro-Woven,Air Jordan 15 Retro Woven\r\n921194 011 FF,BLACK/BLACK-WHITE,Mens,PSNY - Friends and Family,515.0,[],[],air-jordan-15-retro-woven_921194-011-ff.png,[air-jordan-15-retro-woven_921194-011-ff_2.png...,https://www.stadiumgoods.com/jordan-air-jordan...,None,Jordan 15
3,921194 011,Jordan,Air Jordan 15 Retro PSNY,Jordan,Jordan,Jordan_Air-Jordan-15-Retro-PSNY,The good folks at Public School return with th...,BLACK/BLACK-SAIL-BLACK,Mens,PSNY,210.0,"[Rubber, Suede]","[15 to Present, Jordan]",air-jordan-15-retro-psny_921194-011.png,"[air-jordan-15-retro-psny_921194-011_2.png, ai...",https://www.stadiumgoods.com/jordan-air-jordan...,Jordan,Jordan 15
4,921194 011a,Jordan,Air Jordan 15 Retro PSNY,Jordan,Jordan,Jordan_Air-Jordan-15-Retro-PSNY,The good folks at Public School return with th...,BLACK/BLACK-WHITE,Mens,PSNY Friends and Family,460.0,[Synthetics],"[15 to Present, Jordan, Gifts]",air-jordan-15-retro-psny_921194-011a.png,"[air-jordan-15-retro-psny_921194-011a_2.png, a...",https://www.stadiumgoods.com/jordan-air-jordan...,Jordan,Jordan 15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5253,BD7927,Adidas,ZX 4000 4D,None,Adidas,Adidas_ZX-4000-4D,ZX 4000 4D\r\nBD7927,GRETHR/CBLACK/HIREYE,Mens,Grey/High Res Yellow,515.0,[],[],zx-4000-4d_bd7927.png,"[zx-4000-4d_bd7927_2.png, zx-4000-4d_bd7927_3....",https://www.stadiumgoods.com/adidas-zx-4000-4d...,None,4D
5254,BD7865,Adidas,ZX 4000 4D,None,Adidas,Adidas_ZX-4000-4D,The adidas ZX 4000 4D “Carbon” combines the vi...,CARBON/CARBON/CARBON,Mens,,295.0,[],[],zx-4000-4d_bd7865.png,"[zx-4000-4d_bd7865_2.png, zx-4000-4d_bd7865_3....",https://www.stadiumgoods.com/adidas-zx-4000-4d...,None,4D
5255,B42203,Adidas,ZX 4000 4D,None,Adidas,Adidas_ZX-4000-4D,The adidas ZX 4000 4D “Red One/Core Black/Brig...,FTWWHT/GRETWO/LINGRN,Mens,Grey,399.0,[],[],zx-4000-4d_b42203.png,"[zx-4000-4d_b42203_2.png, zx-4000-4d_b42203_3....",https://www.stadiumgoods.com/adidas-zx-4000-4d...,None,4D
5256,G26783,Adidas,I x 4D,None,Adidas,Adidas_I-x-4D,The adidas 4D-5923 “Never Made Pack” combines ...,RED/FTWWHT/RED,Mens,Never Made Pack,0.0,[],[],i-x-4d_g26783.png,"[i-x-4d_g26783_2.png, i-x-4d_g26783_3.png, i-x...",https://www.stadiumgoods.com/adidas-i-x-4d-nev...,None,4D


Replace on better base model names:

In [119]:
with open("../meta/base-model-map.json", "r") as stream:
    model_map = json.load(stream)

gdf["basemodel"] = gdf["basemodel"].map(model_map).fillna(gdf["basemodel"])
pd.unique(gdf["basemodel"].values)

array(['Jordan 15', 'Jordan 14', 'Jordan 13', 'Jordan 12', 'Jordan 11',
       'Jordan 10', 'Jordan 1', 'Jordan 2', 'Jordan 3', 'Jordan 4',
       'Jordan 5', 'Jordan 6', 'Jordan 7', 'Jordan 8', 'Jordan 9',
       'Jordan Westbrook', 'Jordan New School', 'Jordan Future',
       'Jordan Spizike', 'Jordan', 'Jordan Son Of Mars',
       'Jordan Why Not Zer0', 'LeBron', 'Kobe Bryant', 'Kyrie Irving',
       'Cortez', 'Penny Hardaway', 'Dunk High', 'Dunk Mid', 'Dunk Low',
       'SB Blazer', 'React Element', 'Foamposite', 'Air Force 1',
       'Air Max 720', 'Air Max 270', 'Air Max 95', 'Air Max 97',
       'Air Max 98', 'Air Max 90', 'Air Max 180', 'Air Max 360',
       'Air Max Plus', 'Air Max Tailwind', 'Air Max 1', 'Air Fear Of God',
       'Vapormax', 'Air Max', 'Air Trainer', 'Air Zoom',
       'Air More Uptempo', 'Lunarepic', 'Huarache', 'Metcon', 'Mercurial',
       'Blazer', 'Tiempo', 'Pegasus', 'Phantom', 'Presto', 'Roshe',
       'Internationalist', 'React', 'Stefan Janoski', 'Ul

Generate brands and models data frames:

In [ ]:
brands_set = set(pd.unique(rdf["brand"].values))
all_brands = set(pd.read_json("../meta/brands.json")[0].T.to_dict().values())
brands_list = list(brands_set.union(all_brands))
brands_list